In [1]:
import numpy as np
import pandas as pd
import pickle
import janestreet

from time import time

from multiprocessing.pool import ThreadPool
from tensorflow.keras.models import load_model

In [2]:
generation = 19
member = 5

f_mean = pickle.load(open('../input/last-{}-{}/fmean_{}.pkl'.format(generation,member,member),'rb'))
model = load_model('../input/last-{}-{}/model_{}.h5'.format(generation,member,member))
scalers = pickle.load(open('../input/last-{}-{}/scalers_{}.pkl'.format(generation,member,member),'rb'))
f_notused = pickle.load(open('../input/last-{}-{}/fnotused_{}.pkl'.format(generation,member,member),'rb'))
description = pickle.load(open('../input/last-{}-{}/description_{}.pkl'.format(generation,member,member),'rb'))

In [3]:
#Model description:

print(description)

Train size: 1

Lower limit for earning: 0

Number of hiden layers: 19

Activation on input layer: elu

Number of neurons on input layer: 191

Activation on output layer: linear

Number of neurons on hidden layers: [ 80  92  77 217 219 199  61  83 189 122 241  31  21 190 232  13 110 103
 131]

Activation on hidden layers: ['elu' 'selu' 'softmax' '0' 'elu' 'softplus' 'relu' 'elu' 'elu'
 'hard_sigmoid' 'exponential' 'selu' 'linear' 'elu' 'relu' 'linear'
 'linear' 'relu' 'elu']

Features used: [  1   3   4   5   6   7   8  10  13  17  18  19  21  22  25  26  27  29
  30  32  33  34  35  38  39  40  43  44  47  49  50  51  52  53  58  59
  61  62  63  64  65  68  70  71  73  74  75  77  79  82  83  84  85  87
  89  91  92  93  94  95  97  98  99 100 104 105 109 111 113 116 117 120
 121 122 127 128 129]

Scalers of the features: [StandardScaler(with_mean=False, with_std=False), QuantileTransformer(output_distribution='normal', random_state=5), StandardScaler(), PowerTransformer(standardize=F

In [4]:
env = janestreet.make_env()
iter_test = env.iter_test()
f = len(scalers)
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.drop(columns=f_notused)
    
    if (test_df.weight.all() <= 0):
        sample_prediction_df.action = 0
        env.predict(sample_prediction_df)
    else:
        test_df = test_df.drop(columns=['weight']).values
            
        if(np.isnan(test_df).any()):
            test_df = np.nan_to_num(test_df) + np.isnan(test_df) * f_mean
        
        
        for scaler,i in zip(scalers,range(f)):
            if (scaler!='passthrough'):
                test_df[:,i:i+1] = scaler.transform(test_df[:,i:i+1])
        
        
        sample_prediction_df.action = int(np.median(model(test_df,training=False).numpy()) > 0.5)
        env.predict(sample_prediction_df)